In [87]:
!pip install spotipy

In [114]:
import base64
import requests
import datetime
from urllib.parse import urlencode
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

spotify_api_key = "6b68c3794bbb4c1cbf3ec4a8f2f3f6c3"
spotify_api_key_secret = "73f5aa7628f7443190a6515ff45c602e"


class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        # In case we want to inherit from somehwere else
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {"Authorization": f"Basic {client_creds_b64}"}

    def get_token_data(self):
        return {"grant_type": "client_credentials"}

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data["access_token"]
        expires_in = data["expires_in"]
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers

    # def get_resource(self, lookup_id, resource_type="albums", version="v1"):
    #     endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
    #     headers = self.get_resource_header()
    #     r = requests.get(endpoint, headers=headers)
    #     if r.status_code not in range(200, 299):
    #         return {}
    #     return r.json()

    # def get_artist(self, _id):
    #     return self.get_resource(_id, resource_type="artists")

    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        resp = requests.get(lookup_url, headers=headers)
        if resp.status_code not in range(200, 299):
            return {}
        return resp.json()

    def search(
        self, query=None, operator=None, operator_query=None, search_type="artist"
    ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k, v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        return self.base_search(query_params)

In [115]:
auth_manager = SpotifyClientCredentials(spotify_api_key, spotify_api_key_secret)

In [116]:
spotipy_instance = spotipy.Spotify(auth_manager=auth_manager)

In [117]:
def retrieve_spotify_id(song_name, artist_name):
    """
    Example:

    song_id = retrieve_spotify_id('margaritaville','jimmy buffett')

    print("Song ID is:", song_id)

    Output:
    Song ID is: 4EEjMyQub6tgFVshlM9j1M

    """

    spotify = SpotifyAPI(spotify_api_key, spotify_api_key_secret)

    token = spotify.get_access_token()
    if token == None:
        return "Failed to get Spotify token"
    else:
        try:
            song = spotify.search(
                {"track": song_name, "artist": artist_name}, search_type="track"
            )
            song_id = song["tracks"]["items"][0]["id"]
        except:
            return "There was an error finding your song. Please enter another song."

    return song_id

In [167]:
spotify = SpotifyAPI(spotify_api_key, spotify_api_key_secret)
song_data = spotify.search(
                {"track": "Margaritaville", "artist": "Jimmy Buffet"}, search_type="track"
            )



release_date = song_data['tracks']['items'][0]['album']['release_date']



In [172]:
import datetime
format = '%Y-%m-%d'
year = datetime.datetime.strptime(release_date, format).year
year

1977

In [155]:
auth_manager = SpotifyClientCredentials(spotify_api_key, spotify_api_key_secret)
spotipy_instance = spotipy.Spotify(auth_manager=auth_manager)

audio_features = spotipy_instance.audio_features(tracks=[song_id])
audio_features

[{'acousticness': 0.711,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4EEjMyQub6tgFVshlM9j1M',
  'danceability': 0.8,
  'duration_ms': 250933,
  'energy': 0.545,
  'id': '4EEjMyQub6tgFVshlM9j1M',
  'instrumentalness': 0.00191,
  'key': 2,
  'liveness': 0.104,
  'loudness': -13.637,
  'mode': 1,
  'speechiness': 0.0357,
  'tempo': 124.943,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/4EEjMyQub6tgFVshlM9j1M',
  'type': 'audio_features',
  'uri': 'spotify:track:4EEjMyQub6tgFVshlM9j1M',
  'valence': 0.861}]

In [151]:
for i in rec_list['tracks']:
  album_list.append(i['album']['name'])
  artist_list.append(i['artists'][0]['name'])
  song_list.append(i['name'])

KeyError: ignored

In [148]:
song_list

['Take Me Home, Country Roads',
 'Say Hey (I Love You) (feat. Cherine Tanya Anderson)',
 "We Didn't Start the Fire",
 'You Might Think',
 'Casey Jones - 2013 Remaster',
 'Sunshine (Go Away Today)',
 'One Week',
 'When the Sun Goes Down',
 'Life in the Fast Lane - 2013 Remaster',
 'Toes',
 'Still the One',
 "It's Still Rock and Roll to Me",
 'Caught Up In You',
 'Cherry Bomb',
 'The Sound Of Sunshine',
 'Night Moves',
 'Centerfield',
 'Something To Talk About',
 'Werewolves of London - 2007 Remaster',
 'Hook']

In [133]:
rec_list['tracks'][0]['artists'][0]['name']

'John Denver'

In [139]:
rec_list['tracks'][0]['name']

'Take Me Home, Country Roads'

In [ ]:
spotify_id = retrieve_spotify_id("Margaritaville", "Jimmy Buffet")

In [ ]:
spotify = SpotifyAPI(spotify_api_key, spotify_api_key_secret)

token = spotify.get_access_token()
if token == None:
  print("Failed to get Spotify token")
song_data = spotify.search({"track": song_name, "artist": artist_name}, search_type="track")
song_data

In [ ]:
spotify_id

'7adzk3KQw3ll2GiiTFS0xd'

In [ ]:
spotify = SpotifyAPI(spotify_api_key, spotify_api_key_secret)

In [ ]:
spotify.get_access_token()

'BQAcijpblHls9gAQYvaOOavpHKwuFtItF1MiHn56yYsDO-5w-UsF8Epa-CTxSEFx-HUl5vz7_PhBzZ0ko2Q'

In [ ]:
audio_features = spotipy_instance.audio_features(tracks=[spotify_id])

In [ ]:
spotipy_instance.recommendations(seed_artists=)

In [ ]:
audio_features_dict = audio_features

danceability = audio_features_dict[0]["danceability"]
energy = audio_features_dict[0]["energy"]
key = audio_features_dict[0]["key"]
loudness = audio_features_dict[0]["loudness"]
mode = audio_features_dict[0]["mode"]
speechiness = audio_features_dict[0]["speechiness"]
acousticness = audio_features_dict[0]["acousticness"]
instrumentalness = audio_features_dict[0]["instrumentalness"]
liveness = audio_features_dict[0]["liveness"]
valence = audio_features_dict[0]["valence"]
tempo = audio_features_dict[0]["tempo"]
duration_ms = audio_features_dict[0]["duration_ms"]
time_signature = audio_features_dict[0]["time_signature"]

user_song_features = [
    danceability,
    energy,
    key,
    loudness,
    mode,
    speechiness,
    acousticness,
    instrumentalness,
    liveness,
    valence,
    tempo,
    duration_ms,
    time_signature,
]

In [60]:
user_song_features_df = pd.DataFrame([user_song_features], columns=['danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature'])
user_song_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.8,0.545,2,-13.637,1,0.0357,0.711,0.00191,0.104,0.861,124.943,250933,4


In [ ]:
song_name = "Margaritaville"
artist_name = "Jimmy Buffet"
spotify_id

In [ ]:
import pandas as pd
spotify_df = pd.read_csv('smaller_tracks_features.csv')

In [ ]:
if spotify_id in spotify_df['id'].values:
  i = spotify_df[spotify_df['id'] == spotify_id].index
  spotify_df = spotify_df.drop(i)
  print("Dropped")
else:
  print(False)

Dropped


In [ ]:
len(spotify_df)

199999

In [ ]:
spotify_df.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')

In [ ]:
cols_to_drop = ['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'year', 'release_date']

spotify_df = spotify_df.drop(cols_to_drop, axis=1)
spotify_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.901,0.1970,9,-10.862,1,0.1220,0.234000,0.00000,0.1500,0.6710,128.493,146467,4
1,0.325,0.9860,7,-4.148,1,0.1810,0.000098,0.03240,0.3470,0.1520,123.153,445867,4
2,0.650,0.9970,4,-3.627,0,0.0918,0.062500,0.88800,0.4800,0.6330,127.086,362547,4
4,0.265,0.6520,2,-12.279,0,0.0541,0.116000,0.72100,0.0923,0.1150,170.520,256720,4
5,0.403,0.1180,0,-21.256,1,0.0361,0.984000,0.95700,0.0945,0.3840,130.710,455533,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.793,0.5620,4,-8.586,0,0.3390,0.139000,0.00000,0.0944,0.3410,144.985,121707,4
199996,0.114,0.9360,6,-3.389,1,0.0936,0.000006,0.00019,0.3220,0.2050,191.625,159659,4
199997,0.112,0.0158,9,-21.892,0,0.0422,0.970000,0.94400,0.0871,0.0964,71.745,219133,5
199998,0.777,0.6640,8,-5.785,0,0.3170,0.052500,0.01480,0.3440,0.9370,95.001,185480,4


In [44]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [42]:
def euclidian_distance(point1, point2):
  point1 = np.array(point1)
  point2 = np.array(point2)
  dist = np.linalg.norm(point1 - point2)
  return dist

In [81]:
def get_10_recs(input_song_normalized, spotify_df_normalized, spotify_df):
  spotify_df['sim'] = ""
  for i, features in enumerate(spotify_df_normalized):
    spotify_df.at[i, 'sim'] = euclidian_distance(input_song_normalized, features)
  return spotify_df

In [173]:
scaler = MinMaxScaler()
sample_df = spotify_df.sample(100)
spotify_df_normalized = scaler.fit_transform(sample_df[['danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature']])

In [174]:
spotify_df_normalized[0]

array([0.79275254, 0.50196243, 0.72727273, 0.70310847, 1.        ,
       0.06318325, 0.54672969, 0.        , 0.04079991, 0.63842533,
       0.21637836, 0.25698845, 0.75      ])

In [175]:
input_song_normalized = scaler.transform(user_song_features_df)
input_song_normalized

array([[0.9081658 , 0.55012656, 0.18181818, 0.63687856, 1.        ,
        0.01926996, 0.71457036, 0.00198958, 0.06217676, 0.88660676,
        0.42206416, 0.2771195 , 0.75      ]])

In [176]:
get_10_recs(input_song_normalized, spotify_df_normalized, sample_df)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,sim
94540,6q5fB6uX5q5wJfyatUqxh1,Milagre Do Povo,Mina D'Água do Meu Canto,2O2G16pLz3vBVDzTTItiQq,['Gal Costa'],['1b8kpp4DUwt1hWaxTiWQhD'],6.0,1.0,False,0.707,0.498,8.0,-11.696,1.0,0.0594,0.544,0.000000,0.0854,0.629,98.166,235333.0,4.0,1995.0,6/14/1995,
27345,1ox5R1TWQG76s9xOTRiuPN,Keep Me From Blowin' Away,In My View,131mIW0o9TSw4ywDuKlmHH,['Charlie Sizemore'],['5qhPkelajWSgPVUhZdGagf'],13.0,1.0,False,0.677,0.109,7.0,-17.716,1.0,0.0326,0.709,0.000000,0.1540,0.224,89.241,177133.0,3.0,2011.0,10/30/2011,
44453,3MHxfij2Giwe1Ibxy74MP9,Introduction,The Transcontinental,20MFNrUGTnV2Z7YRze7jFO,"['Roc C', 'Not Dr. Trevis']","['5bGlerwfdc1PN2Igf9HAYC', '4Wer3336qAism1yz9I...",1.0,1.0,True,0.400,0.149,3.0,-19.544,1.0,0.0708,0.809,0.000000,0.3560,0.227,123.018,150760.0,3.0,2009.0,5/19/2009,
42011,6NPRlbDoDm1a2A8mzZiIWq,Nothing Compares 2 U - Live,Don't Explain (Live),4mltX3Xaf99gWDx1RvFzaZ,"['Heinz Sauer', 'Michael Wollny']","['1kBzT6LReLozVTUJMYOu76', '7MbjIv3XpMc2ciNcoW...",2.0,1.0,False,0.376,0.439,9.0,-10.050,1.0,0.1620,0.977,0.360000,0.7240,0.384,72.677,309427.0,5.0,2012.0,11/30/2012,
9918,4f6zWxVdKNwO37dreh42kX,Valse Des Vachers,Times Ain't Like They Used To Be Vol. 8: Early...,7xyp8aCAZS09QYD6ZnoOnC,['Dennis McGee'],['1q2QwNdlkGCIJMzUea5sqI'],17.0,1.0,False,0.550,0.483,1.0,-12.709,1.0,0.0389,0.976,0.000071,0.2180,0.876,96.740,164213.0,4.0,2005.0,6/20/2005,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8289
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.67509
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.30505
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33393
